<div style="display:flex; align-items:center; background-color:black; padding:20px;">
  <span style="font-size:3em; color:red; font-weight:bold; margin-right:30px;">Predicting the English Premier League</span>
  <img src="logo.png" alt="logo" style="max-width:120px; height:auto; border-radius:10px; box-shadow:0 0 10px #222; background:white;">
</div>


<font size="5.5" color="green">Introduction about the Project</font>


The most popular and developed sport in the world is Football, and arguably the English Premier league is the most competitive, and it is therefore the most natural choice to study. The game of football is constantly evolving, and features that may be important in one generation may become less so later - there has never been an analytic definition of the game. This is where machine learning could prove invaluable.

In this project, we look at applying statistical and machine learning methods, in order to attempt at predicting match results based on historic data of the teams that play the match. A majority part of the project builds a framework that takes raw match data and creates suitable features based on historic statistics to match results. We then by using Decision trees and Random forest classifiers and finally we build deep learning logic, combine it with a number of feature selection tools, including PCA, to get the best accuracy metric.

<font size="5.5" color="green">Problem Statement</font>

The premise then lies, to build a machine learning framework, that can use historic data from football matches between two teams and learn how to best predict outcomes of games. The general outline of the steps to be taken to get to a framework that can get to the premise are as follows:                                                                   
* Recover a source of data with enough information that links team performance to final result.
* Build a tool which takes the raw data and transforms it into a format in which each match has a number of historic features based on previous games, such as 'Average goals in x games'.
* Use feature selection tools to filter out unimportant/redundant features.
* Feed features into a number of machine learning algorithms and decide on the best one to fine-tune.
* Fine-tune the chosen machine learning algorithm for the greatest accuracy.

<font size="5.5" color="green">Background & Metrics</font>